In [1]:
import numpy as np  
import matplotlib.pyplot as plt
import gymnasium as gym


## Create and Inspect

In [5]:
env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True, render_mode="ansi")

print("State space:", env.observation_space.n)
print("Action space:", env.action_space.n)

# Reset the environment before rendering and display the initial map
obs, info = env.reset()
render_output = env.render()
print(render_output)


State space: 64
Action space: 4

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG

